In [11]:
import json
import os
from statistics import mean
from collections import defaultdict
import pandas as pd

# 初始化一个字典来存储每个 model 的数据
model_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

def parse_filename(file_name):
    """从文件名中解析 defender、model_name、attacker、num_prompts 和 time_str"""
    parts = file_name.split('_')
    defender = parts[0]  # 第一部分是 defender
    model_name = parts[1]
    attacker = parts[2]
    num_prompts = parts[3]
    time_str = parts[4]
    return defender, model_name, attacker, num_prompts, time_str

def process_folder(folder_path, folder):
    gpt_scores = []
    
    # 找到以 _safe_eval.json 结尾的文件
    for file in os.listdir(folder_path):
        if file.endswith('_safe_eval.json'):
            safe_eval_file = os.path.join(folder_path, file)
            with open(safe_eval_file, 'r') as f:
                data = json.load(f)
                
                processed_count = 0  # 用于记录成功处理的条目数

            if isinstance(data, list):
                # 遍历每个 result
                for item in data:
                    try:
                        parsed_result = item.get("parsed_result", {})
                        if parsed_result is None:
                            parsed_result = {}

                        # 获取每类分数，确保其为数值类型
                        helpfulness_score = parsed_result.get("helpfulness", {}).get("score", 0)
                        clarity_score = parsed_result.get("clarity", {}).get("score", 0)
                        factuality_score = parsed_result.get("factuality", {}).get("score", 0)
                        depth_score = parsed_result.get("depth", {}).get("score", 0)
                        engagement_score = parsed_result.get("engagement", {}).get("score", 0)
                        
                          
                        
                        # 确保分数为数字，如果是字符串则尝试转换为浮点数
                        helpfulness_score = float(helpfulness_score) if isinstance(helpfulness_score, (int, float, str)) and helpfulness_score else 0
                        clarity_score = float(clarity_score) if isinstance(clarity_score, (int, float, str)) and clarity_score else 0
                        factuality_score = float(factuality_score) if isinstance(factuality_score, (int, float, str)) and factuality_score else 0
                        depth_score = float(depth_score) if isinstance(depth_score, (int, float, str)) and depth_score else 0
                        engagement_score = float(engagement_score) if isinstance(engagement_score, (int, float, str)) and engagement_score else 0


                            # 如果任何分数为 0，跳过该条目
                        # if any(score == 0 for score in [helpfulness_score, clarity_score, factuality_score, depth_score, engagement_score]):
                        #     print("跳过因分数为 0 的条目")
                        #     continue
                        # 添加到总分列表
                        gpt_scores.append({
                            "helpfulness": helpfulness_score,
                            "clarity": clarity_score,
                            "factuality": factuality_score,
                            "depth": depth_score,
                            "engagement": engagement_score
                        })

                        processed_count += 1  # 成功处理的条目数加 1
                    except Exception as e:
                        print(f"跳过异常条目: {e}")
                        continue

            # 输出处理的总条数
            print(f"成功处理的条目总数: {processed_count}")
    
    # 计算每类分数的平均值和总平均分
    if gpt_scores:
        avg_helpfulness = mean([x["helpfulness"] for x in gpt_scores])
        avg_clarity = mean([x["clarity"] for x in gpt_scores])
        avg_factuality = mean([x["factuality"] for x in gpt_scores])
        avg_depth = mean([x["depth"] for x in gpt_scores])
        avg_engagement = mean([x["engagement"] for x in gpt_scores])
        
        overall_average = mean([avg_helpfulness, avg_clarity, avg_factuality, avg_depth, avg_engagement])
        
        print(f"平均 helpfulness 分数: {avg_helpfulness}")
        print(f"平均 clarity 分数: {avg_clarity}")
        print(f"平均 factuality 分数: {avg_factuality}")
        print(f"平均 depth 分数: {avg_depth}")
        print(f"平均 engagement 分数: {avg_engagement}")
        print(f"总体平均分: {overall_average}")
        
        defender, model_name, attacker, num_prompts, time_str = parse_filename(folder)
        # 保存到 model_data
        model_data[model_name][defender][attacker].append({
            
            "avg_helpfulness": avg_helpfulness,
            "avg_clarity": avg_clarity,
            "avg_factuality": avg_factuality,
            "avg_depth": avg_depth,
            "avg_engagement": avg_engagement,
            "overall_average": overall_average
        })

# Function to create the summary table
def create_summary_table(data):
    # 创建一个空的数据框
    columns = set()
    rows = set()
    
    # 首先收集所有的 defender 和 attacker 名称
    for defender, attackers in data.items():
        rows.add(defender)
        for attacker in attackers.keys():
            columns.add(attacker)
    
    # 创建一个 pandas DataFrame
    table = pd.DataFrame(index=list(rows), columns=list(columns))
    
    # 填充表格数据
    for defender, attackers in data.items():
        for attacker, entries in attackers.items():
            cell_data = []
            for entry in entries:
                scores = entry.get("scores", [])
                if scores:
                    for score in scores:
                        cell_data.append(f"Helpfulness: {score['avg_helpfulness']}\nClarity: {score['avg_clarity']}\nFactuality: {score['avg_factuality']}\nDepth: {score['avg_depth']}\nEngagement: {score['avg_engagement']}\nOverall: {score['overall_average']}")
            
            # 将数据放到表格的单元格里
            table.at[defender, attacker] = "\n".join(cell_data)
    
    return table

# 遍历主文件夹中的所有子文件夹并处理
base_dir = '/home/kz34/Yang_Ouyang_Projects/ICLR2025/jailbreaking_related/SafeDecoding/exp_outputs'

for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        if folder == 'previous_result':
            continue
        if "Just-Eval" not in folder:
            continue 
        print(f"处理文件夹: {folder}")
        process_folder(folder_path, folder)

# 为每个 model_name 创建并保存表格
for model_name, defenders in model_data.items():
    summary_table = create_summary_table(defenders)
    print(f"Model: {model_name}")
    print(summary_table)
    
    # 将结果保存为 csv 文件
    summary_table.to_csv(f'{model_name}_defender_justeval_summary.csv')

处理文件夹: SafeDecoding_mistral_Just-Eval_1000_2024-10-14 16:29:58
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳过因分数为 0 的条目
跳

In [12]:
model_data

defaultdict(<function __main__.<lambda>()>,
            {'mistral': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'SafeDecoding': defaultdict(list,
                                      {'Just-Eval': [{'avg_helpfulness': 4.789693593314763,
                                         'avg_clarity': 4.8314763231197775,
                                         'avg_factuality': 4.685236768802229,
                                         'avg_depth': 4.410863509749303,
                                         'avg_engagement': 4.119777158774373,
                                         'overall_average': 4.567409470752089}]}),
                          'nodefense': defaultdict(list,
                                      {'Just-Eval': [{'avg_helpfulness': 4.646,
                                         'avg_clarity': 4.894,
                                         'avg_factuality': 4.709,
                                         'avg_depth': 4.358,
   